In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
import json
with open('/content/drive/MyDrive/manya/Convolve project/train.json', 'r') as f:
    train_data = json.load(f)

df = pd.DataFrame.from_dict(train_data,orient='index')
df1 = pd.read_csv('/content/drive/MyDrive/manya/Convolve project/test.csv')
print(df.shape)
print(df1.shape)

(4152659, 1)
(595300, 2)


In [5]:
df.head()

,0
1117838570 2005.06.03 R02-M1-N0-C:J12-U11 2005-06-03-15.42.50.363779 R02-M1-N0-C:J12-U11 RAS KERNEL INFO instruction cache parity error corrected\n,normal
1117838570 2005.06.03 R02-M1-N0-C:J12-U11 2005-06-03-15.42.50.527847 R02-M1-N0-C:J12-U11 RAS KERNEL INFO instruction cache parity error corrected\n,normal
1117838570 2005.06.03 R02-M1-N0-C:J12-U11 2005-06-03-15.42.50.675872 R02-M1-N0-C:J12-U11 RAS KERNEL INFO instruction cache parity error corrected\n,normal
1117838570 2005.06.03 R02-M1-N0-C:J12-U11 2005-06-03-15.42.50.823719 R02-M1-N0-C:J12-U11 RAS KERNEL INFO instruction cache parity error corrected\n,normal
1117838570 2005.06.03 R02-M1-N0-C:J12-U11 2005-06-03-15.42.50.982731 R02-M1-N0-C:J12-U11 RAS KERNEL INFO instruction cache parity error corrected\n,normal


In [6]:
df1.head()

,ID,Log
0,0,1124336301 2005.08.17 R13-M1-N8-C:J12-U01 200...
1,1,1118553175 2005.06.11 R30-M0-N9-C:J16-U01 200...
2,2,1118536033 2005.06.11 R30-M0-N9-C:J16-U01 200...
3,3,1117992566 2005.06.05 R30-M1-N6-C:J03-U11 200...
4,4,1118538965 2005.06.11 R30-M0-N9-C:J16-U01 200...


#**DATA** **PREPARATION**

In [7]:
df.reset_index(inplace=True)

In [8]:
df.rename(columns={"index":"Log"},inplace=True)

In [9]:
df.head()

,Log,0
0,1117838570 2005.06.03 R02-M1-N0-C:J12-U11 200...,normal
1,1117838570 2005.06.03 R02-M1-N0-C:J12-U11 200...,normal
2,1117838570 2005.06.03 R02-M1-N0-C:J12-U11 200...,normal
3,1117838570 2005.06.03 R02-M1-N0-C:J12-U11 200...,normal
4,1117838570 2005.06.03 R02-M1-N0-C:J12-U11 200...,normal


In [10]:
df.isnull().sum()

Log    0
0      0
dtype: int64

In [11]:
# Remove leading space and trailing \n and numerical data from each log
df.Log = df.Log.apply(lambda x:" ".join(str(x).split(' ')[6:]).split('\n')[0])
df1[' Log'] = df1[' Log'].apply(lambda x:" ".join(str(x).split(' ')[6:]).split('\n')[0])

In [12]:
df.head()

,Log,0
0,RAS KERNEL INFO instruction cache parity error...,normal
1,RAS KERNEL INFO instruction cache parity error...,normal
2,RAS KERNEL INFO instruction cache parity error...,normal
3,RAS KERNEL INFO instruction cache parity error...,normal
4,RAS KERNEL INFO instruction cache parity error...,normal


In [13]:
df1.head()

,ID,Log
0,0,RAS KERNEL FATAL rts: kernel terminated for re...
1,1,RAS KERNEL FATAL data TLB error interrupt
2,2,RAS KERNEL FATAL data TLB error interrupt
3,3,RAS KERNEL INFO generating core.6463
4,4,RAS KERNEL FATAL data TLB error interrupt


In [14]:
df.duplicated().sum()

3881720

In [15]:
df = df.drop_duplicates()
print(df.shape)

(270939, 2)


## **NLP**

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
%%time
import re
import nltk
from nltk.tokenize import word_tokenize
import string
from nltk.stem import WordNetLemmatizer

def prepare_text(s):
    # Step 1: Tokenization
    s = word_tokenize(s) # only tokenize words
    #display(f"Tokenized words: {s}")
    # Step 2: Remove Punctuation in python
    s = [token for token in s if token not in string.punctuation]
    #display(f"Python based removal: {s}")
    # Step 3: Remove Stop Words
    stop_words = nltk.corpus.stopwords.words('english')
    s = [token for token in s if token not in stop_words]
    #display(f"nltk text without stop words: {s}")
    # Step 4: Select only alphanumeric words
    s = [token for token in s if token.isalnum()]
    # # Step 5: Remove single and double character words
    # s = [token for token in s if len(s)>2]
    # Step 6: Lemmatizer 
    lemmatizer = WordNetLemmatizer()
    s = [lemmatizer.lemmatize(token) for token in s]
    # Step 7: Convert list of tokens to string
    s = " ".join(s)
    # Step 8: Convert string to lower case
    s = s.lower()
      
    return s

df['Log'] = df['Log'].apply(str) # convert each row to string
df['Log'] = df['Log'].apply(prepare_text)

df1[' Log'] = df1[' Log'].apply(str) 
df1[' Log'] = df1[' Log'].apply(prepare_text)

CPU times: user 4min 3s, sys: 14.9 s, total: 4min 18s
Wall time: 4min 18s


In [ ]:
df.columns=['Log','label']

In [ ]:
df.to_csv('/content/drive/MyDrive/manya/Convolve project/df_processed.csv', index='False')
df1.to_csv('/content/drive/MyDrive/manya/Convolve project/df1_processed.csv', index='False')

In [16]:
df = pd.read_csv('/content/drive/MyDrive/manya/Convolve project/df_processed.csv')
df1 = pd.read_csv('/content/drive/MyDrive/manya/Convolve project/df1_processed.csv')
print(df.shape)
print(df1.shape)

(270939, 3)
(595300, 3)


In [17]:
print(df.columns)
print(df1.columns)

Index(['Unnamed: 0', 'Log', 'label'], dtype='object')
Index(['Unnamed: 0', 'ID', ' Log'], dtype='object')


In [18]:
df.drop('Unnamed: 0',axis=1,inplace=True)
df1.drop('Unnamed: 0',axis=1,inplace=True)

In [19]:
# Vectorizing Text to Int Sequence using Bag of Words(BoW): Count Vectorizer

from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(stop_words='english',
                     #ngram_range=(1,1),
                     min_df=5,
                     max_df=0.8)
# min_df = a defines to remove low freq words that occur only in a % docu. 
#max_df = b meaning, ignore words appeared in b% of the documents as they are too common """min_df=.002,max_df=0.8"""
X_vec = cv.fit_transform(df['Log'])
X1_vec = cv.transform(df1[' Log'])
print(X_vec.shape)
print(X1_vec.shape)
print(cv.vocabulary_)
# Save vectorizer.vocabulary_
import pickle
filename = "/content/drive/MyDrive/manya/Convolve project/count_vectorizer_vocabulary_processed.pkl" 
pickle.dump(cv.vocabulary_, open(filename,"wb"))

(270939, 1651)
(595300, 1651)
{'instruction': 1442, 'cache': 1286, 'parity': 1532, 'error': 1375, 'corrected': 1316, 'linkcard': 1480, 'midplaneswitchcontroller': 1499, 'performing': 1535, 'bit': 1267, 'sparing': 1599, 'generating': 1421, 'ddr': 1341, 'detected': 1346, 'rank': 1559, 'symbol': 1615, 'l3': 1475, 'edram': 1370, 'dcr': 1339, '0x0157': 163, 'ce': 1293, 'sym': 1614, '0x0b8580c0': 195, 'mask': 1490, '0x10': 204, 'total': 1627, 'activating': 1161, 'redundant': 1566, 'steering': 1607, 'failure': 1399, 'consider': 1309, 'replacing': 1568, 'card': 1288, '13': 299, '25': 529, '14': 314, '0x20': 212, '16': 351, '12': 286, '17': 366, '0xd0': 225, '0xa0': 222, '0xc0': 224, '0x80': 220, '0x30': 215, '0x08': 190, '0x0b85ec80': 196, '0x07': 189, '0x0b85ee80': 197, '0x0c': 201, '0x0b85f2a0': 198, '0x0b85f680': 199, '0x0e': 203, '0x0b85f880': 200, '0x04': 186, '0x02': 167, '0x03': 185, '0x40': 216, '0x70': 219, '0x1b858ec0': 207, '0x1b859080': 208, '0x1b859480': 209, '0xb0': 223, '0x0a': 

In [20]:
import pickle
pickle.load(open("/content/drive/MyDrive/manya/Convolve project/count_vectorizer_vocabulary_processed.pkl","rb"))

{'instruction': 1442,
 'cache': 1286,
 'parity': 1532,
 'error': 1375,
 'corrected': 1316,
 'linkcard': 1480,
 'midplaneswitchcontroller': 1499,
 'performing': 1535,
 'bit': 1267,
 'sparing': 1599,
 'generating': 1421,
 'ddr': 1341,
 'detected': 1346,
 'rank': 1559,
 'symbol': 1615,
 'l3': 1475,
 'edram': 1370,
 'dcr': 1339,
 '0x0157': 163,
 'ce': 1293,
 'sym': 1614,
 '0x0b8580c0': 195,
 'mask': 1490,
 '0x10': 204,
 'total': 1627,
 'activating': 1161,
 'redundant': 1566,
 'steering': 1607,
 'failure': 1399,
 'consider': 1309,
 'replacing': 1568,
 'card': 1288,
 '13': 299,
 '25': 529,
 '14': 314,
 '0x20': 212,
 '16': 351,
 '12': 286,
 '17': 366,
 '0xd0': 225,
 '0xa0': 222,
 '0xc0': 224,
 '0x80': 220,
 '0x30': 215,
 '0x08': 190,
 '0x0b85ec80': 196,
 '0x07': 189,
 '0x0b85ee80': 197,
 '0x0c': 201,
 '0x0b85f2a0': 198,
 '0x0b85f680': 199,
 '0x0e': 203,
 '0x0b85f880': 200,
 '0x04': 186,
 '0x02': 167,
 '0x03': 185,
 '0x40': 216,
 '0x70': 219,
 '0x1b858ec0': 207,
 '0x1b859080': 208,
 '0x1b85948

In [21]:
# Tranforming Sparse Int Vectors to Dense Float Vectors using TF-IDF
from sklearn.feature_extraction.text import TfidfTransformer
tfr = TfidfTransformer(use_idf = True)
X_vec = tfr.fit_transform(X_vec).toarray().astype(np.float32)
X1_vec = tfr.transform(X1_vec).toarray().astype(np.float32)
print(X_vec.shape)
print(X1_vec.shape)

(270939, 1651)
(595300, 1651)


In [22]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
df['label']=le.fit_transform(df['label'])
print(df.shape)

(270939, 2)


In [23]:
le.classes_

array(['abnormal', 'normal'], dtype=object)

In [24]:
X=X_vec
X1=X1_vec
Y=df['label']

## MACHINE LEARNING MODELS

In [25]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.2) 
print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)

(216751, 1651) (54188, 1651) (216751,) (54188,)


In [25]:
!pip install flaml

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [26]:
# Auto ML : Fast and Light Auto ML FLAML (Microsoft)
from flaml import AutoML
automl = AutoML()
automl.fit(X_train,y_train,time_budget= 60,
           metric = 'f1',task = 'classification',eval_method='cv', n_splits=10,
           sample_weight_val = 'balanced'
          )
#print(automl.model.estimator)
best_model = automl.model#best model

[flaml.automl.automl: 02-05 13:20:55] {2715} INFO - task = classification
[flaml.automl.automl: 02-05 13:20:55] {2717} INFO - Data split method: stratified
[flaml.automl.automl: 02-05 13:20:55] {2720} INFO - Evaluation method: cv
[flaml.automl.automl: 02-05 13:20:56] {2847} INFO - Minimizing error metric: 1-f1
[flaml.automl.automl: 02-05 13:20:56] {2993} INFO - List of ML learners in AutoML Run: ['lgbm', 'rf', 'xgboost', 'extra_tree', 'xgb_limitdepth', 'lrl1']
[flaml.automl.automl: 02-05 13:20:56] {3322} INFO - iteration 0, current learner lgbm
[flaml.automl.automl: 02-05 13:21:31] {3460} INFO - Estimated sufficient time budget=347847s. Estimated necessary time budget=8018s.
[flaml.automl.automl: 02-05 13:21:31] {3507} INFO -  at 35.9s,	estimator lgbm's best error=0.0000,	best estimator lgbm's best error=0.0000
[flaml.automl.automl: 02-05 13:21:31] {3322} INFO - iteration 1, current learner xgboost
[flaml.automl.automl: 02-05 13:22:04] {3507} INFO -  at 68.9s,	estimator xgboost's best 

In [28]:
import pickle
pickle.dump(best_model,open('/content/drive/MyDrive/manya/Convolve project/decision_tree_model.pkl','wb'))

In [27]:
dt_model = pickle.load(open('/content/drive/MyDrive/manya/Convolve project/decision_tree_model.pkl','rb'))

In [29]:
y_pred_dt = dt_model.predict(X_test)

In [30]:
# Confusion Matrix
from sklearn.metrics import confusion_matrix
print("Confusion Matrix:\n")
print(confusion_matrix(y_test,y_pred_dt))

Confusion Matrix:

[[  424     0]
 [    0 53764]]


In [31]:
# Classification report
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_dt))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       424
           1       1.00      1.00      1.00     53764

    accuracy                           1.00     54188
   macro avg       1.00      1.00      1.00     54188
weighted avg       1.00      1.00      1.00     54188



Gaussian Naive Bayes

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import RepeatedStratifiedKFold
model_gnb = GaussianNB()
param_grid = {'var_smoothing': np.logspace(0,-9, num=10)} 
cv_method = RepeatedStratifiedKFold(n_splits=5,  n_repeats=2)
grid_search_NB = GridSearchCV(estimator=model_gnb,param_grid=param_grid, cv=cv_method,verbose=10,scoring='f1')
grid_search_NB.fit(X,Y)

Fitting 10 folds for each of 10 candidates, totalling 100 fits
[CV 1/10; 1/10] START var_smoothing=1.0.........................................
[CV 1/10; 1/10] END ..........var_smoothing=1.0;, score=0.996 total time=   6.1s
[CV 2/10; 1/10] START var_smoothing=1.0.........................................
[CV 2/10; 1/10] END ..........var_smoothing=1.0;, score=0.996 total time=   4.9s
[CV 3/10; 1/10] START var_smoothing=1.0.........................................
[CV 3/10; 1/10] END ..........var_smoothing=1.0;, score=0.996 total time=   4.0s
[CV 4/10; 1/10] START var_smoothing=1.0.........................................
[CV 4/10; 1/10] END ..........var_smoothing=1.0;, score=0.996 total time=   3.8s
[CV 5/10; 1/10] START var_smoothing=1.0.........................................
[CV 5/10; 1/10] END ..........var_smoothing=1.0;, score=0.996 total time=   3.8s
[CV 6/10; 1/10] START var_smoothing=1.0.........................................
[CV 6/10; 1/10] END ..........var_smoothing=1.

GridSearchCV(cv=RepeatedStratifiedKFold(n_repeats=2, n_splits=5, random_state=None),
             estimator=GaussianNB(),
             param_grid={'var_smoothing': array([1.e+00, 1.e-01, 1.e-02, 1.e-03, 1.e-04, 1.e-05, 1.e-06, 1.e-07,
       1.e-08, 1.e-09])},
             scoring='f1', verbose=10)

In [33]:
pickle.dump(grid_search_NB, open('/content/drive/MyDrive/manya/Convolve project/naive_bayes_model',"wb"))

In [32]:
NB_model = pickle.load(open('/content/drive/MyDrive/manya/Convolve project/naive_bayes_model',"rb"))

In [33]:
y_pred_gnb = NB_model.predict(X_test)

In [34]:
# Confusion Matrix
from sklearn.metrics import confusion_matrix
print("Confusion Matrix:\n")
print(confusion_matrix(y_test,y_pred_gnb))

Confusion Matrix:

[[  424     0]
 [    0 53764]]


In [35]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_gnb))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00       424
           1       1.00      1.00      1.00     53764

    accuracy                           1.00     54188
   macro avg       1.00      1.00      1.00     54188
weighted avg       1.00      1.00      1.00     54188



In [ ]:
y_pred = NB_model.predict(X1)
print(y_pred.shape)

In [ ]:
y_pred = le.inverse_transform(y_pred)
print(y_pred.shape)

In [ ]:
submission = pd.read_csv('/content/drive/MyDrive/manya/Convolve project/sample_submission.csv')
print(submission.shape)

In [ ]:
submission[' Label']=y_pred

In [ ]:
submission

In [ ]:
submission.to_csv('/content/drive/MyDrive/manya/Convolve project/final_submission.csv',index='False')